In [1]:
%reload_ext snakeviz
%run 'MC.py'

In [162]:
lattice(4).sites

[<0,0,0>,
 <0,1,0>,
 <0,2,0>,
 <0,3,0>,
 <1,0,0>,
 <1,1,0>,
 <1,0,1>,
 <1,1,1>,
 <2,0,0>,
 <2,0,1>,
 <2,0,2>,
 <2,0,3>]

In [261]:
group(3).chi

[[3, 2, 2, 1, 1, 2],
 [2, 3, 1, 2, 2, 1],
 [2, 1, 3, 2, 2, 1],
 [1, 2, 2, 3, 1, 2],
 [1, 2, 2, 1, 3, 2],
 [2, 1, 1, 2, 2, 3]]

In [128]:
class model(object):
    def __init__(self):
        self.dof = 0 # onsite degrees of freedom
        self.Nsite = 0 # number of sites
        self.config = [] # configuration
        self.chi = numpy.array([[0]]) # chi table, as numpy array
        self.H = [[]] # Hamiltonian in row compressed storage
        self.route = [] # update route
        self.boundary = [] # boundary sites
    # one step of MC update
    def MCstep(self):
        # each MC step goes over the update route once
        for i in self.route:
            h = sum(J*self.chi[self.config[j]] for j, J in self.H[i])
            w = numpy.exp(h)
            self.config[i] = numpy.random.choice(self.dof,p=w/sum(w))
    # calculate energy of the system by config
    def energy(self):
        energy = 0
        for i in range(self.Nsite):
            chii = self.chi[self.config[i]]
            energy -= sum(J*chii[self.config[j]] for j, J in self.H[i])
        return energy
class hypertree_model(model):
    def __init__(self, L=0, dof=0, Js=(1.,1.), branches=2):
        # setup the lattice
        latt = lattice(L, branches=branches)
        # setup the permutation group
        G = group(dof)
        # initialization
        self.dof = dof
        self.Nsite = latt.Nsite
        self.config = [0]*latt.Nsite
        # chi is adjusted by dof
        self.chi = numpy.array(G.chi)-self.dof
        # build Hamiltonian
        self.H = [[] for i in range(latt.Nsite)]
        for i in range(latt.Nsite):
            a = latt.sites[i] # take site i
            bs = a.IR # get its IR sites
            for b, J in zip(bs, Js):
                j = b.index
                self.H[i].append((j,J))
                self.H[j].append((i,J))
        # assign update route (use the sublattice structure)
        self.route = []
        for key in ('odd','even'):
            for a in latt.sublattice[key]:
                self.route.append(a.index)
        # assign boundary sites
        self.boundary = [a.index for a in latt.sublattice['boundary']]

In [129]:
sys = hypertree_model(4,2,(1.,0.1))
print(list(range(sys.Nsite)))
for step in range(10):
    sys.MCstep()
    print(sys.config, sys.energy())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1] 2.6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] 0.4
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1] 6.6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 0.4
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1] 3.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] 0.4
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] 0.8
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0] 4.4
[0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0] 8.8


In [5]:
def test():
    sys = hypertree_model(128,2,(1.,0.1))
    energies = []
    for step in range(32):
        sys.MCstep()
        energies.append(sys.energy())
%snakeviz test()

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmp8o5xy2ge'. 


 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmp7so6qwno'. 
